# Sử dụng model phobert-base cho bài toán regress tính khuynh hướng giá stock dựa trên tin tức

### Load data from google sheet 
https://docs.google.com/spreadsheets/d/1ZSLM2gc6GBWs1fvvNwzA0D7jusRWv2xPVqOVoXcYJ3o/edit#gid=732292548

In [1]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
# define the scope
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('./questioniso-0a60ce8c39c1.json', scope)

# authorize the clientsheet 
client = gspread.authorize(creds)

sheet = client.open_by_key('1ZSLM2gc6GBWs1fvvNwzA0D7jusRWv2xPVqOVoXcYJ3o')

# get the first sheet of the Spreadsheet
sheet_instance = sheet.worksheet('news')

data = sheet_instance.get_all_records()
print('data', data[:1])

data [{'id': 1, 'title': 'Dow Jones tăng hơn 300 điểm, chấm dứt 5 phiên giảm liên tiếp', 'content': 'Chứng khoán Mỹ khởi sắc vào ngày thứ Ba (14/3), khi nhà đầu tư đặt cược vào việc nguy cơ lan truyền sau vụ phá sản của các ngân hàng Silicon Valley và Signature đã được ngăn chặn.\n\n\n\nKết thúc phiên giao dịch ngày thứ Ba, chỉ số Dow Jones tăng 336.26 điểm (tương đương 1.06%) lên 32,155.40 điểm, chấm dứt chuỗi 5 phiên lao dốc liên tiếp. Chỉ số S&P 500 tiến 1.65% lên 3,919.29 điểm. Chỉ số Nasdaq Composite cộng 2.14% lên 11,428.15 điểm.\n\nSự nhiệt tình mua vào các cổ phiếu ngân hàng của nhà đầu tư đã giảm đi phần nào vào buổi chiều. Tuy nhiên, nhiều cổ phiếu vẫn ghi nhận đà tăng, đánh dấu bước đảo chiều sau 2 phiên giảm sâu khi nhà đầu tư ngày càng tin tưởng rằng những ngân hàng đó sẽ không chịu chung số phận như ngân hàng Silicon Valley và Signature. Các nhà quản lý cho biết vào ngày 12/3 rằng họ đã xây dựng một kế hoạch để can thiệp cho tất cả những người gửi tiền ở 2 ngân hàng này.\

In [2]:
df = pd.DataFrame()
title = []
content = []
date = []
stock_price = []
status = []

for values in data:
    title.append(values['title'])
    content.append(values['content'])
    date.append(values['date'])
    stock_price.append(values['stock_price'])
    status.append(values['status'])

df['title'] = title
df['content'] = content
df['date'] = date
df['stock_price'] = stock_price
df['status'] = status

df[0:5]

,title,content,date,stock_price,status
0,"Dow Jones tăng hơn 300 điểm, chấm dứt 5 phiên ...",Chứng khoán Mỹ khởi sắc vào ngày thứ Ba (14/3)...,,100.0,1.0
1,Vàng thế giới lùi nhẹ khi lợi suất trái phiếu ...,Giá vàng giảm vào ngày thứ Ba (14/3) khi lợi s...,,98.0,-0.5
2,Xét xử vụ DongABank: 'Lạnh gáy' với khoản vay ...,Sau khi bị cáo Phùng Ngọc Khánh (cựu Chủ tịch ...,,95.0,-1.0
3,Cựu tổng giám đốc Ngân hàng Đông Á Trần Phương...,Cùng các đồng phạm gây thiệt hại tổng số tiền ...,,93.0,-1.0
4,Nóng: Ngân hàng Nhà nước giảm lãi suất điều hành,Ngân hàng Nhà nước (NHNN) vừa ban hành các quy...,,94.0,1.0


In [3]:
import pandas as pd
from datetime import date, timedelta
import re
from underthesea import word_tokenize, sent_tokenize
from gensim import corpora, models

def clean_text(text):
    text = str(text).lower() # lowercase text
    return text

df = df[df['stock_price'] != '']

documents = df['content'].values.tolist()
print('documents:', documents[0:1])
print('len:', len(documents))

documents: ['Chứng khoán Mỹ khởi sắc vào ngày thứ Ba (14/3), khi nhà đầu tư đặt cược vào việc nguy cơ lan truyền sau vụ phá sản của các ngân hàng Silicon Valley và Signature đã được ngăn chặn.\n\n\n\nKết thúc phiên giao dịch ngày thứ Ba, chỉ số Dow Jones tăng 336.26 điểm (tương đương 1.06%) lên 32,155.40 điểm, chấm dứt chuỗi 5 phiên lao dốc liên tiếp. Chỉ số S&P 500 tiến 1.65% lên 3,919.29 điểm. Chỉ số Nasdaq Composite cộng 2.14% lên 11,428.15 điểm.\n\nSự nhiệt tình mua vào các cổ phiếu ngân hàng của nhà đầu tư đã giảm đi phần nào vào buổi chiều. Tuy nhiên, nhiều cổ phiếu vẫn ghi nhận đà tăng, đánh dấu bước đảo chiều sau 2 phiên giảm sâu khi nhà đầu tư ngày càng tin tưởng rằng những ngân hàng đó sẽ không chịu chung số phận như ngân hàng Silicon Valley và Signature. Các nhà quản lý cho biết vào ngày 12/3 rằng họ đã xây dựng một kế hoạch để can thiệp cho tất cả những người gửi tiền ở 2 ngân hàng này.\n\nChứng chỉ quỹ SPDR S&P Regional Banking ETF tiến 2%, phục hồi phần nào sau đà lao dốc

### Tính trung bình word2vec để tạo ra mảng training chỉ 2 chiều:
(dựa theo: https://github.com/PradipNichite/Youtube-Tutorials/blob/main/Yotutube_WordVectors.ipynb)

In [133]:
import numpy as np
y_train = df['status'].values.tolist()
statusNp = np.asarray(df['status'].values.tolist())
Y_train = np.expand_dims(statusNp, axis = 1)
print(Y_train.shape, Y_train)

(19, 1) [[ 1. ]
 [-0.5]
 [-1. ]
 [-1. ]
 [ 1. ]
 [-1. ]
 [-1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [-1. ]
 [ 0.9]
 [-1. ]
 [-1. ]
 [-1. ]
 [ 0.5]
 [ 0.1]
 [ 0.4]]


In [5]:
import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

Some weights of the model checkpoint at vinai/phobert-base-v2 were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens

In [6]:
print(np.zeros((2, 5)))

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [145]:
from torch.nn.functional import pad
def trim_words(sentence, num_word = 200):
    # Split the sentence into words
    words = sentence.split()
    # Trim the sentence to 5 words
    trimmed_sentence = " ".join(words[:num_word])
    return trimmed_sentence

max_len = 400
def vectorize(sentence):
    sentence_token = trim_words(word_tokenize(sentence, format="text"))
    input_ids = torch.tensor([tokenizer.encode(sentence_token)])
    features = phobert(input_ids)  # Models outputs are now tuples
    features_np = features.last_hidden_state.detach()
    padding_length = max_len - len(features_np[0])
    padded_a = pad(features_np, (0, 0, 0, padding_length), value=0, mode='constant')
    # words_vecs = np.array(words_vecs)
    return padded_a.detach().numpy()[0]

def vectorize_arr(sentences):
    t = np.array([vectorize(sentence) for sentence in sentences])
    return t

text = """
Chứng khoán Mỹ khởi sắc vào ngày thứ Ba (14/3), khi nhà đầu tư đặt cược vào việc nguy cơ lan truyền sau vụ phá sản của các ngân hàng Silicon Valley và Signature đã được ngăn chặn.
Kết thúc phiên giao dịch ngày thứ Ba, chỉ số Dow Jones tăng 336.26 điểm (tương đương 1.06%) lên 32,155.40 điểm, chấm dứt chuỗi 5 phiên lao dốc liên tiếp. Chỉ số S&P 500 tiến 1.65% lên 3,919.29 điểm. Chỉ số Nasdaq Composite cộng 2.14% lên 11,428.15 điểm.
Sự nhiệt tình mua vào các cổ phiếu ngân hàng của nhà đầu tư đã giảm đi phần nào vào buổi chiều. Tuy nhiên, nhiều cổ phiếu vẫn ghi nhận đà tăng, đánh dấu bước đảo chiều sau 2 phiên giảm sâu khi nhà đầu tư ngày càng tin tưởng rằng những ngân hàng đó sẽ không chịu chung số phận như ngân hàng Silicon Valley và Signature. Các nhà quản lý cho biết vào ngày 12/3 rằng họ đã xây dựng một kế hoạch để can thiệp cho tất cả những người gửi tiền ở 2 ngân hàng này.
Chứng chỉ quỹ SPDR S&P Regional Banking ETF tiến 2%, phục hồi phần nào sau đà lao dốc 12% trong phiên trước đó. Cổ phiếu First Republic Bank bứt phá gần 27% sau khi bốc hơi gần 62% vào ngày 13/3. Cổ phiếu KeyCorp vọt gần 7% trong đợt phục hồi nhẹ sau khi trượt dốc 27%.
"""

# (1, 239, 768)
print('documents:', vectorize(text).shape)
X_train = vectorize_arr(documents)

documents: (400, 768)


In [126]:
print(X_train)

[[[ 0.10691808  0.56538063 -0.0462637  ... -0.02383143  0.30707052
   -0.12627435]
  [-0.15620075  0.06405588 -0.4220878  ...  0.08413314  0.1673638
   -0.06000924]
  [-0.16119361  0.29304975 -0.06564619 ...  0.06615663 -0.04387109
   -0.00355058]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 0.09067404  0.48833108 -0.12780698 ...  0.03539366  0.4004186
   -0.03919914]
  [-0.04390531  0.17930356 -0.20418605 ...  0.34756333  0.08095603
   -0.4143583 ]
  [ 0.2092944   0.03868642 -0.1207267  ...  0.2061632   0.01764482
   -0.62089825]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 0.02782983  0.7210824  -0.2140605 

In [220]:
import tensorflow as tf
from keras.layers import MaxPooling1D, Flatten, Dense, LSTM, Bidirectional, GlobalAveragePooling1D
import numpy as np

# Create a sample 1D input data
# x = [[[1], [3], [5], [2]]]

x = tf.constant([1., 2., 3., 4., 5., 8., 12., 7.])
x = tf.reshape(x, [1, -1, 1])

# Create a MaxPooling1D layer
maxpool_layer = MaxPooling1D(pool_size=2, strides=8, padding='valid')

print('input shape:', X_train.shape)
# Apply the MaxPooling1D layer to the input data
output_data = maxpool_layer(X_train)

# Print the output
print(output_data.shape)


input shape: (19, 400, 768)
(19, 50, 768)


In [110]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM

# Define the model
model = Sequential()
model.add(Embedding(768, 512, input_shape=(400, 768)))
model.add(LSTM(128, dropout=0.2))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.predict(X_train)

ValueError: Input 0 of layer "lstm_13" is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, 400, 768, 512)

In [221]:
from keras.models import Sequential
from keras.layers import MaxPooling1D, Flatten, Dense, LSTM, Bidirectional, GlobalAveragePooling1D, Input, Average, LayerNormalization

input_shape = (400, 768)
model = Sequential()
model.add(Input(shape=input_shape))
model.add(Bidirectional(LSTM(
        100, 
        return_sequences = True, 
        recurrent_dropout=0.2)))
model.add(LayerNormalization())
model.add(Dense(2, activation='relu'))
model.add(Dense(1))
model.add(MaxPooling1D(pool_size=2, strides=8, padding='valid'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.summary()
results = model.predict(X_train)
print(results)

Model: "sequential_54"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_43 (Bidirecti  (None, 400, 200)         695200    
 onal)                                                           
                                                                 
 layer_normalization_6 (Laye  (None, 400, 200)         400       
 rNormalization)                                                 
                                                                 
 dense_36 (Dense)            (None, 400, 2)            402       
                                                                 
 dense_37 (Dense)            (None, 400, 1)            3         
                                                                 
 max_pooling1d_24 (MaxPoolin  (None, 50, 1)            0         
 g1D)                                                            
                                                     

In [222]:

model.fit(X_train, Y_train, epochs=15, batch_size=10)

Epoch 1/15
2/2 [==============================] - 5s 557ms/step - loss: 0.9750 - mae: 0.9010
Epoch 2/15
2/2 [==============================] - 1s 797ms/step - loss: 0.8147 - mae: 0.8632
Epoch 3/15
2/2 [==============================] - 2s 752ms/step - loss: 0.8147 - mae: 0.8632
Epoch 4/15
2/2 [==============================] - 1s 739ms/step - loss: 0.8147 - mae: 0.8632
Epoch 5/15
2/2 [==============================] - 2s 878ms/step - loss: 0.8147 - mae: 0.8632
Epoch 6/15
2/2 [==============================] - 1s 686ms/step - loss: 0.8147 - mae: 0.8633
Epoch 7/15
2/2 [==============================] - 2s 779ms/step - loss: 0.8146 - mae: 0.8632
Epoch 8/15
2/2 [==============================] - 2s 621ms/step - loss: 0.8146 - mae: 0.8633
Epoch 9/15
2/2 [==============================] - 1s 719ms/step - loss: 0.8146 - mae: 0.8633
Epoch 10/15
2/2 [==============================] - 1s 643ms/step - loss: 0.8146 - mae: 0.8633
Epoch 11/15
2/2 [==============================] - 1s 568ms/step - lo

In [226]:
test_data = [
    "Cùng các đồng phạm gây thiệt hại tổng số tiền trên 5.500 tỉ đồng, cựu tổng giám đốc Ngân hàng Đông Á Trần Phương Bình tiếp tục bị đề nghị truy tố trong vụ án thứ 4.",
    "Công ty Lọc hóa dầu Bình Sơn (BSR) lên kế hoạch bảo dưỡng lớn cho nhà máy lọc dầu Dung Quất, đề ra mức lãi giảm gần 90%.",
    "Chứng khoán Mỹ khởi sắc vào ngày thứ Ba (14/3), khi nhà đầu tư đặt cược vào việc nguy cơ lan truyền sau vụ phá sản của các ngân hàng Silicon Valley và Signature đã được ngăn chặn.",
    "Nhà đầu tư mới của Bamboo Airways sẽ hỗ trợ tài chính cho hãng và giúp ông Trịnh Văn Quyết một khoản tiền để khắc phục hậu quả vụ án (nếu có)."
]

test_data_c = vectorize_arr(test_data)
print('test_data_c:', test_data_c.shape)
result = model.predict(test_data_c)
print('result:\n', result[2])

test_data_c: (4, 400, 768)
1/1 [==============================] - 0s 106ms/step
result:
 [[-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]
 [-0.00495339]]
